In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import libconfig as cfg
import json
import yaml

In [3]:
[x for x in dir(cfg) if not x.startswith("_")]

['config',
 'evaluator',
 'get_option',
 'get_option_default',
 'get_option_description',
 'lock_option',
 'register_option',
 'reset_option',
 'reset_options',
 'set_option',
 'set_options_from_JSON',
 'set_options_from_YAML',
 'set_options_from_dict',
 'show_options',
 'write_options_to_JSON',
 'write_options_to_YAML']

In [4]:
cfg.register_option("number", "integer", 5, "int", "this is an integer example", True)
cfg.register_option("number", "float", 3.4, "float", "this is a float example", False)
cfg.register_option("no_number", "boolean", True, "bool", "this is a boolean example", False)
cfg.register_option("no_number", "text", "the_string", "string", "this is a string example", False)
cfg.register_option("paths", "in", "demo.ipynb", "path_in", "this is a path example that has to exist", False)
cfg.register_option("paths", "out.json", "configs.json", "path_out",
                    "this is a path example that does not have to exist", False)
cfg.register_option("paths", "out.yaml", "configs.yaml", "path_out",
                    "this is a path example that does not have to exist", False)
cfg.show_options()

,primary-key,secondary-key,value,type,default,locked,description
0,number,integer,5,int,5,True,this is an integer example
1,number,float,3.4,float,3.4,False,this is a float example
2,no_number,boolean,1,bool,1,False,this is a boolean example
3,no_number,text,the_string,string,the_string,False,this is a string example
4,paths,in,demo.ipynb,path_in,demo.ipynb,False,this is a path example that has to exist
5,paths,out.json,configs.json,path_out,configs.json,False,this is a path example that does not have to exist
6,paths,out.yaml,configs.yaml,path_out,configs.yaml,False,this is a path example that does not have to exist


In [5]:
try:
    cfg.set_option( "number", "integer", 5.5 )
except ValueError, e:
    print e

Value must have type '<type 'int'>'


In [6]:
try:
    cfg.set_option( "number", "random", 5.5 )
except KeyError, e:
    print e

'number.random option is not registered'


In [7]:
cfg.set_option( "number", "float", 5.5 )
cfg.show_options()

,primary-key,secondary-key,value,type,default,locked,description
0,number,integer,5,int,5,True,this is an integer example
1,number,float,5.5,float,3.4,False,this is a float example
2,no_number,boolean,1,bool,1,False,this is a boolean example
3,no_number,text,the_string,string,the_string,False,this is a string example
4,paths,in,demo.ipynb,path_in,demo.ipynb,False,this is a path example that has to exist
5,paths,out.json,configs.json,path_out,configs.json,False,this is a path example that does not have to exist
6,paths,out.yaml,configs.yaml,path_out,configs.yaml,False,this is a path example that does not have to exist


In [8]:
cfg.write_options_to_JSON( cfg.get_option( "paths", "out.json" ) )
cfg.reset_option( "number", "float" )
cfg.show_options()

,primary-key,secondary-key,value,type,default,locked,description
0,number,integer,5,int,5,True,this is an integer example
1,number,float,3.4,float,3.4,False,this is a float example
2,no_number,boolean,1,bool,1,False,this is a boolean example
3,no_number,text,the_string,string,the_string,False,this is a string example
4,paths,in,demo.ipynb,path_in,demo.ipynb,False,this is a path example that has to exist
5,paths,out.json,configs.json,path_out,configs.json,False,this is a path example that does not have to exist
6,paths,out.yaml,configs.yaml,path_out,configs.yaml,False,this is a path example that does not have to exist


In [9]:
try:
    cfg.set_options_from_JSON( cfg.get_option( "paths", "out.json" ) )
except ValueError, e:
    print e

number.integer option is locked


In [10]:
cfg.set_option( "number", "float", 12.0 )
cfg.write_options_to_YAML( cfg.get_option( "paths", "out.yaml" ) )
cfg.reset_option( "number", "float" )
cfg.show_options()

,primary-key,secondary-key,value,type,default,locked,description
0,number,integer,5,int,5,True,this is an integer example
1,number,float,3.4,float,3.4,False,this is a float example
2,no_number,boolean,True,bool,1,False,this is a boolean example
3,no_number,text,the_string,string,the_string,False,this is a string example
4,paths,in,demo.ipynb,path_in,demo.ipynb,False,this is a path example that has to exist
5,paths,out.json,configs.json,path_out,configs.json,False,this is a path example that does not have to exist
6,paths,out.yaml,configs.yaml,path_out,configs.yaml,False,this is a path example that does not have to exist


In [11]:
try:
    cfg.set_options_from_YAML( cfg.get_option( "paths", "out.yaml" ) )
except ValueError, e:
    print e

number.integer option is locked


In [69]:
cfg.reset_options()